##### Imports:

In [19]:
from utils import separate_for_training, preprocess
import pickle
from chroma import get_chromagram
import pandas as pd

##### Steps:

1. Training / Testing Data Split
2. Create Chromagram from Training Data
3. Create HMM Initialization Components
    a. Initial State Probabilities
    b. Transition Probability Matrix
    c. Mu Value
    d. Emission Matrix
4. Create HMM Object
5. Fit / Train HMM

##### Training / Test Data Split:

In [20]:
# Load data and split into training and test
piece_name_dict = preprocess('dataset.pkl')

training_piece_names, _, test_piece_names = separate_for_training(piece_name_dict, 0.8, 0.)

with open(r"dataset.pkl", 'rb') as data:
    midi_data = pickle.load(data)

100%|██████████| 5762/5762 [00:02<00:00, 2574.01it/s]


##### Create Chromagram from Training Data:

In [21]:
song_chromagrams = []
for song_name in training_piece_names:
    indiv_chroma = get_chromagram(song_name, midi_data)

chromagram = pd.concat(song_chromagrams)
chromagram.head(6)

TypeError: get_chromagram() takes 1 positional argument but 2 were given